## 6 - RAII (or: stack-based resource management)
##### **Author: Adam Gatt**

### The resource management pattern
A common pattern that repeatedly comes up in software development goes as follows:
1. You need to acquire a resource
2. You do something with the resource
3. You have to release the resource or perform some other clean up

This is the [resource management](https://en.wikipedia.org/wiki/Resource_management_(computing)) pattern, and it is a generalisation of a wide range of usage patterns, including the following:

| Acquire the resource | Activity | Release / cleanup |
| --- | --- | --- |
| Memory allocation (new) | **Using dynamic memory** | Freeing the memory (delete) |
| Opening a file | **Performing file operations** | Closing the file |
| Connecting to a database server | **Making database transactions** | Closing the connection |
| Acquire a mutex/lock | **Perform thread-safe activity** | Release the mutex/lock |
| Altering a bitmap mask | **Performing drawing operations** | Restoring the previous mask |
| Establish an SFTP connection | **Perform a file transfer** | Closing the connection |
| [Turn off system interrupts](https://energia.nu/reference/en/language/functions/interrupts/nointerrupts/) | **Perform an atomic operation** | [Re-establish interrupts](https://energia.nu/reference/en/language/functions/interrupts/interrupts/) |

The third step must be important if an activity is to fall into this pattern. The programmer must be careful to ensure that it occurs or else there will be important consequences, such as memory leaks, or a thread deadlock, or exhausting of a database connection pool.

We can decide that the problem is solved by making sure that whenever an acquisition occurs we can write a matching release operation. But as with anything we decide to enforce manually, complications get in the way:
 * A function may have unexpected early returns that require us to write multiple release statements (and check each one to determine if the resource was ever actually acquired, and check to make sure we don't release it twice).
 * An exception may be thrown that prevents the program from ever reaching our release statement (and without checked exceptions we might not realise that a statement might throw an exception).
 * The resource might only need to be acquired based on some condition, and so we must be careful to ensure that releases are executed the exact same amount as the executing acquisitions.
 * If the function is long and messy enough we might simply forget the release statement and not notice its absence.

### What do other languages do?
We might notice that memory allocation isn't too much of an issue with interpreted or virtual machine languages which can employ [garbage collection](https://en.wikipedia.org/wiki/Garbage_collection_(computer_science)) to clean up old memory. Although this comes at a cost to overhead and reliability, it allows us to write entire programs in Java or Python without ever needing to manually delete an object or even consider when their deletion will occur.

But similar automated schemes are usually not available for these other resource management activities. How can a virtual machine know when to close down a database connection, for example? Even if we have some Database object that falls out of use, how do we know that deleting it will close the connection? And even if it did, how do we know if the garbage collector will be called soon enough or if the connection will remain unclosed for quite some time, using up a valuable limited resource?

And if the VM follows some other heuristic (e.g. after some period of inactivity) then how do we cope with the times when the VM gets it wrong? Do we need to check that the connection is fine every single time that we want to make a transaction, and re-establish the connection if it isn't?

Instead, these newer languages often offer language semantics to automatically handle the resource releasing for us. This often involves representing the activity's context as an object that understands how to perform its own cleanup, and then offering syntax in the language itself to trigger that cleanup operation when control leaves that context.
#### C#
C# offers the [using syntax](https://docs.microsoft.com/en-us/dotnet/csharp/language-reference/keywords/using-statement) to automatically manage objects with the [IDisposable](https://docs.microsoft.com/en-us/dotnet/api/system.idisposable?view=net-5.0) interface:

In [ ]:
using (DbConnection conn = new DbConnection())
{
    // Perform database operations
}

#### Python
The Python [with statement](https://www.geeksforgeeks.org/with-statement-in-python/) allows us to call any function that returns a [context manager](https://book.pythontips.com/en/latest/context_managers.html#implementing-a-context-manager-as-a-class) object. It will give us a handle to the object for our own use and call its `__exit__()` method automatically when control flow leaves the scope of the with statement:

In [ ]:
for filename in config_files:
    with File(filename, 'r') as file:
        loaded_configs[filename] = list(file)

### The destructor, C++'s secret weapon
Out of all of the strengths of C++, it has been said that its greatest feature is [precisely determined object lifetimes](https://akrzemi1.wordpress.com/2013/07/18/cs-best-feature/) and more specifically, the destructor.
 * Garbage collected languages do not allow us this level of control, where we can precisely reason the exact moment when an object will be deleted.
 * Even better, when our object is stack-based (not dynamic) then the program will automatically delete the object for us the moment it falls out of scope, with no explicit `delete` statement required.
 * Even betterer, the availability of the destructor allows us to write our own code that will be executed at those moments.

This is the heart of [Resource Acquisition is Initialisation](https://en.wikipedia.org/wiki/Resource_acquisition_is_initialization) (RAII), a hideous name for C++'s answer to the resource management pattern. We can trivially create objects that fulfil a "context manager" role, where they acquire a resource in the constructor and release it in the destructor. If the object is on the stack then we can simply use scopes to manage the lifetime of that object, allowing for the "release" operation to be called automatically without the need for any special syntax like the languages above. As such, I like to think of the concept as "stack-based resource management", or perhaps "scope-based resource management".

### The standard RAII object
The minimum template for a RAII object is remarkably simple:

In [1]:
class ResourceManager {
    public:
    ResourceManager() {
        // Acquire the resource
    }
    
    ~ResourceManager() {
        // Free the resource
    }
}

Perhaps we need data members to keep a reference to the resource (e.g. a file handle) or perhaps we don't (e.g. if we are simply calling global functions to turn interrupts on/off). We can also provide other methods to allow for other operations that we might need with the acquired resource. These will be application-dependent. But the bare minimum of a RAII object is:
* It is a class, that
* Obtains the resource in its constructor, and
* Releases the resource in its destructor

Initialisation and resource acquisition are inextricably tied together, and so "Resource Acquisition is Initialisation".

To use the object we instantiate it on the stack within the scope that we need the resource. Here is an example that uses Qt's [QMutexLocker](https://doc.qt.io/qt-5/qmutexlocker.html), which is an RAII calss for automatically acquiring and releasing a mutex:

In [ ]:
class BackBuffer {
    private:
    QImage pixels;
    QMutex pixelMutex;
    
    ...
    
    public:
    /*
      The backbuffer image can be written to and read from by
      different threads and so we use a Mutex to synchronise
      access to our pixels object.
    */
    void drawToBackBuffer(int colIdx, int rowIdx, QRgb colour) {
        // QMutexLocker accepts a reference to our mutex and calls lock() on it
        QMutexLocker locker{&pixelMutex};
        
        // Operations from this point onwards are thread-safe with respect to pixelMutex
        
        
        // At the end of the method "locker" will fall out of scope and its
        // destructor will be called, which will call unlock() on our mutex.
        // Nowhere did we need to call unlock() ourselves.
    }
}

Note how in this example our actual resource, the mutex, existed long-term as a data member in our BackBuffer class. But the RAII object `locker` is not stored anywhere or kept as a data member. Its purpose is to be created on the stack within the function itself, and to die when we want the mutex to be released.

### Resource management through scope blocks

It is important to think about what we are doing here. It's not too important that our resource is being managed inside of an object. What's important is that if the object is stack-based then we are tying the resource to the object's scope, and doing so in a way is automatically triggered and robust to unexpected control flows.

### Robust cleanup

### RAII in std and Qt

### Performance considerations